# 7 ドキュメント型DB（MongoDB）

- **[7.1 NoSQLとRDBMS](#7.1-NoSQLとRDBMS)**
    - [7.1.1 NoSQLとは](#7.1.1-NoSQLとは)
    - [7.1.2 NoSQLとRDBMSの違い](#7.1.2-NoSQLとRDBMSの違い)
<br><br>
- **[7.2 MongoDBの基礎知識](#7.2-MongoDBの基礎知識)** 
    - [7.2.1 MongoDBの概要と特徴](#7.2.1-MongoDBの概要と特徴)
    - [7.2.2 MongoDB使用環境](#7.2.2-MongoDB使用環境)
    - [7.2.3 MongoDBの仕組み](#7.2.3-MongoDBの仕組み)
    - [7.2.4 Mongoシェルの基本操作](#7.2.4-Mongoシェルの基本操作)
    - [7.2.5 MongoDBのデータ型](#7.2.5-MongoDBのデータ型)
<br><br> 
- **[7.3 MongoDBの基本操作](#7.3-MongoDBの基本操作)** 
    - [7.3.1 データの挿入](#7.3.1-データの挿入-&#40;Create&#41;)
    - [7.3.2 データの検索](#7.3.2-データの検索-&#40;Read&#41;)
        - [7.3.2.1 カーソル操作](#7.3.2.1-カーソル操作)
    - [7.3.2 データの更新](#7.3.3-データの更新-&#40;Update&#41;)
    - [7.3.4 データの削除](#7.3.4-データの削除-&#40;Delete&#41;)
    - [7.3.5 データの集計](#7.3.5-データの集計)
<br><br>
- **[7.4 MongoDBのパフォーマンス向上](#7.4-MongoDBのパフォーマンス向上)**
    - [7.4.1 インデックス](#7.4.1-インデックス)
    - [7.4.2 MongoDBの統計・メットリックス](#7.4.2-MongoDBの統計・メットリックス)
    - [7.4.3 データのバックアップ・エクスポート](#7.4.3-データのバックアップ・エクスポート)

***

## 7.1 NoSQLとRDBMS

### 7.1.1 NoSQLとは

第5章および第6章では、従来のリレーショナルデータベースについて学びました。
本章では、いわゆるNoSQL系のデータベースの一つであるMongoDBについて学びます。

従来のRDBMSが明確なデータ構造を定義し、SQL言語による優れた検索性能を持ち、強力なトランザクション機能を備えています。しかしその反面、水平スケーリングしづらく、マスタスレーブ構造のため書き込み負荷が集中しやすく、耐障害性も上げにくいことなどの弱点があります。近年、それらの弱点に対処するのに、生まれたのがNoSQL系データベースです。

NoSQLとは「Not + Only + SQL」の頭文字を取ったもので、一般的にすべての非リレーショナルデータベースを指す単語です。NoSQL系データベースには、ドキュメント型（例:[MongoDB](https://www.mongodb.com/)）、グラフ型（例:[Neo4j](https://neo4j.com/)）、キー値型（例:[Amazon DynamoDB](https://aws.amazon.com/dynamodb/)）、列指向型(例:[Cassandra](http://cassandra.apache.org/)）など、様々なデータモデルが利用されています。

### 7.1.2 NoSQLとRDBMSの違い

前節で触れたスケーラビリティやデータモデルの違いの他に多くの違いがあります。

NoSQLとのRDBMSとの間特に重要な違いの一つとして、データの取得・操作方法が挙げられます。RDBMSでは構造化クエリ言語 (SQL) 準拠のクエリを使用するのに対して、NoSQLの多くはオブジェクト指向のAPIを使用して、データの挿入・検索等を簡単に行うことができます（中にはSQLに似たクエリ言語をサポートするものもあります）。すなわち、NoSQLではデータベースおよび実際のアプリケーションのデータ構造が似ており、RDBMSより扱いやすいことが多いです。

また、NoSQLはRDBMSと違って、データを挿入する前に事前にスキーマを定義・固定する必要がないことが多いです。自由に様々な形式のデータを一緒に保存することが可能で、RDBMSよりフレキシブルです。

逆に、NoSQLの弱点の一つとしては、一般的にトランザクションの機能がないということです。このあと勉強するMongoDBではトランザクション相当の機能を実現できなくもないですが、初心者には少し高度なトピックなので、詳細は割愛します。

***

## 7.2 MongoDBの基礎知識

### 7.2.1 MongoDBの概要と特徴

MongoDBとは現在最も広く利用されているオープンソースのドキュメント型NoSQLデータベースです。従来のRDBMSで使われるテーブル構造ではなく、動的なスキーマを持ったJSONのようななドキュメント（BSON形式）を用いています。
固定スキーマのないBSONを使うことで、一つの記録でで複雑な階層関係を持つデータを格納できたり、必要に応じてフィールドを追加または削除することなどが簡単にできます。そのため、JSON形式を使う多くのWeb APIとの相性がよく、開発がより迅速になると感じる利用者が多いです。

また、MongoDBの豊富なインデックスとデータ構造を使うことで、RDBMSではできない様々なことができます。MongoDBの様々なインデックスを使うことで、位置情報の取扱いを容易にしたり、文字列の検索性能を向上したり、古いデータを自動的に削除したりできます。また、MongoDBの様々なデータ型を使うことで、階層構造のデータを無理に正規化しなくて済みますし、BSONのバイナリ型を使うことで特殊なデータベースシステムを作ることまでできます（例: [GridFS](https://docs.mongodb.com/manual/core/gridfs/)、[Arctic](https://github.com/manahl/arctic)）。

つまり、どちらがより向いているかは最終的には扱っているデータの性質に依りますが、MongoDBはRDBMSよりフレキシブルでパフォーマンスも高いことから、RDBMSが使われているほとんどの場面において代わりにMongoDBを問題なく使用ことが可能です。

### 7.2.2 MongoDB使用環境

MongoDBは一般的には**`Mongoシェル`**と呼ばれるCLIツールを利用するか、自分の好きなプログラミング言語とMongoDBをつなぐ**MongoDBドライバ**と呼ばれるライブリを使うことになります。実際のアプリケーション開発をするときは、基本的にドライバから使うことになりますが、ちょっとしたadminタスク等にはMongoシェルが便利です。また、MongoDBユーザの共通語がシェルで使われるJavascriptなので、本・ネット上のリソースでは基本的にMongoシェル/Javascriptを使った例がほとんどです。

**Mongoシェル**はMongoDBへのインタラクティブなJavaScriptインターフェイスで、データの挿入・検索・更新・削除・集約等の操作の実行ができます。Mongoシェルは、MongoDB本体と一緒にインストールされるので、MongoDBが使える環境であれば、Mongoシェルをコマンドライン<sup id="a1">[1](#f1)</sup>
から`mongo`を実行して使うことができます。しかし、本講座ではJupyter上ですべてが完結するように、特別なJupyter用カーネルを作りました（[IMongo](https://github.com/gusutabopb/imongo)）。よって、本章で紹介するMongoシェル用コマンドは第3章のMySQLコマンドと同様に、そのままJupyter上のコードセルで実行可能になっています。基本的に、シェルの出力をそのまま出力として表示しますが、一部のコマンドの場合、結果をインタラクティブなJSONとして表示しています。カーネルの不具合等があれば[Github issues](https://github.com/gusutabopb/imongo/issues)から連絡をいただければ幸いです。

多くの言語用の**MongoDBドライバ**が[開発されています](https://docs.mongodb.com/ecosystem/drivers/)が、本講座ではPythonを前提としているため、**本章後半**ではPython用のMongoDBドライバである[pymongo](https://github.com/mongodb/mongo-python-driver)を紹介します。

> <sup id="f1">**[1]** CLIが苦手な方には[Robomongo](https://robomongo.org/)など、様々なという[GUIツール](https://docs.mongodb.com/ecosystem/tools/administration-interfaces/)もあります[↩](#a1)</sup>



#### 実行してみよう

前述のように、MongoシェルではJavaScriptを使いますが、Mongoシェルでは普通のJavaScriptには存在しない、特殊なコマンドも[いくつかあります](https://docs.mongodb.com/manual/reference/mongo-shell/#command-helpers)。その一つが`show databases`です。早速実行して、MongoDBが立ち上がっていること確認しましょう。

In [1]:
show databases

admin  0.000GB
local  0.000GB
test   0.000GB

他にのコマンドは`help`や`db.help()`で確認できますが、実際のコマンド操作に入る前に、MongoDBを理解する上で重要な概要について述べます。

### 7.2.3 MongoDBの仕組み

それでは、具体的なコマンドを説明する前に、MongoDBの動作の基本的な概念および仕組みを学びましょう。

* 一つのMongoDBのインスタンスの中には複数の**データベース**を管理することが可能です。ここでいうデータベースとは第5-6章で学んだMySQLと同様にMongoDBにおけるもっとも高レベルなコンテナです。
* 各データベースは一般的に複数の**コレクション**からなります。コレクションは従来のテーブルとほぼ共通しているので、この2つを同じものだと思ってもいいでしょう。
* 各コレクションは複数の**ドキュメント**からなります。先ほどと同様にドキュメントをMySQLでいう**行**と思って構いません。
* 各ドキュメントは1つ以上の**フィールド**からなります。MySQLでいう**列**に近い概念です。
* MongoDBの**インデックス**機能はMySQLのものとよく似ており、インデックスを作ることで、検索・ソート性能を向上させることが可能です。
* MongoDBの**カーソル**もMySQLのものとよく似ており、検索したデータへのポインタのようなものです。MongoDBからデータ取得しようとするときに返されるのは実際のデータではなく、このカーソルであることをと覚えておいてください。

MongoDBでは「テーブル」を「コレクション」と呼ぶなど、一見不必要な新しい用語が登場しますが、これらは実は似て非なるものであることを頭の片隅に入れておいてください。MySQLでは**列**が**テーブル**ごとに定義されるのに対して、MongoDBのようなドキュメント型DBMSでは**フィールド**が**ドキュメント**ごとに定義されている、というのがもっとも根本的な違いです。すなわち、コレクション内の各ドキュメントは独立しており、自由に自身のフィールドを定義することができます。 そのため、**コレクション**は**テーブル**よりシンプルなコンテナでありながら、**ドキュメント**は通常、**行**に比べてより多くの情報を保有します。

### 7.2.4 Mongoシェルの基本操作

Mongoシェルを立ち上げるときに、デフォルトデータベース（通常`test`）に自動的に接続されます。今回はTwitterデータを扱うので、`twitter`というデータベースを選択しましょう。

>#### 使用するデータについて

>これから使うデータはすべてTwitterが提供している[Search API](https://dev.twitter.com/rest/public/search)を使って取得したものです。取得したデータは執筆時点で大きななニュースになっている米国のトランプ新大統領関連のツイートです。

>本節ではMongoDBの基本操作の説明の観点から、APIから取得した生のデータではなく、一部のフィールドを省略したものを使うことにします。しかし、[7.3.5節](#7.3.5-データの集計)以降ではAPIから取得した生データを扱うことにします。

In [2]:
use twitter

switched to db twitter

厳密にいうと、この時点ではデータベースはまだ存在しないですが、最初のコレクションの最初のドキュメントを挿入するときにデータベースが自動的に作られます。コレクションもデータベースと同様に、事前に作る必要はありません。それでは、最初のドキュメントを`tweets`に入れましょう。

In [3]:
db.tweets.insertOne({
  'created_at': 'Sun Feb 05 06:28:27 +0000 2017',
  'id': 828128132402073602,
  'lang': 'ja',
  'text': 'トランプ大統領が招く「憲法の危機」 入国禁止令を停止した裁判官をTwitterで攻撃\n\n困ったちゃん https://t.co/BLWVqXOAAw',
})

{
	"acknowledged" : true,
	"insertedId" : ObjectId("592a0b3248ffa7f5b51f66f0")
}

上記のコマンドは一つの引数を受け取り`tweets`コレクションに対して一つのドキュメントの挿入（`insertOne`）を行います。ここで挿入しているのは特殊なJSONオブジェクトです（Pythonでいう`dict`と同じようなものです）。MongoDBの内部ではそのJSONデータをBSONに変換して保存しています。また、Mongoシェル上では、`db`は常にその時点で使っているデータベース（すなわち、今回は`twitter`）を指します。

先程挿入したドキュメントを読み出すには、コレクションtweetsに対しfindコマンドを使います。<a id='7.2.4-find'></a>

In [4]:
db.tweets.find()

{ "_id" : ObjectId("592a0b3248ffa7f5b51f66f0"), "created_at" : "Sun Feb 05 06:28:27 +0000 2017", "id" : 828128132402073600, "lang" : "ja", "text" : "トランプ大統領が招く「憲法の危機」 入国禁止令を停止した裁判官をTwitterで攻撃\n\n困ったちゃん https://t.co/BLWVqXOAAw" }

MongoDBのコレクションに固定スキーマがないことの特徴を活かすことで、例えば以下のドキュメントも同じ`tweets`コレクションに挿入することができます。

In [5]:
db.tweets.insertOne({
  'created_at': 'Sun Feb 05 06:48:27 +0000 2017',
  'id': 828133165654945792,
  'lang': 'en',
  'text': "@GeorgeTakei She's the best! Comedy will help us make it through the Trump shit show",
  'user_mentions': [{'id': 237845487,
    'name': 'George Takei',
    'screen_name': 'GeorgeTakei'}]
})

{
	"acknowledged" : true,
	"insertedId" : ObjectId("592a0b5b48ffa7f5b51f66f1")
}

ここで、もう一度`find`を実行すると、違うスキーマを持つ2つのドキュメントが同じコレクションに入っていることを確認できます。

In [6]:
db.tweets.find()

{ "_id" : ObjectId("592a0b3248ffa7f5b51f66f0"), "created_at" : "Sun Feb 05 06:28:27 +0000 2017", "id" : 828128132402073600, "lang" : "ja", "text" : "トランプ大統領が招く「憲法の危機」 入国禁止令を停止した裁判官をTwitterで攻撃\n\n困ったちゃん https://t.co/BLWVqXOAAw" }
{ "_id" : ObjectId("592a0b5b48ffa7f5b51f66f1"), "created_at" : "Sun Feb 05 06:48:27 +0000 2017", "id" : 828133165654945800, "lang" : "en", "text" : "@GeorgeTakei She's the best! Comedy will help us make it through the Trump shit show", "user_mentions" : [ { "id" : 237845487, "name" : "George Takei", "screen_name" : "GeorgeTakei" } ] }

### 7.2.5 MongoDBのデータ型

前節では複数のフィールドを持つ2つのツイートを挿入したましたが、それぞれのフィールドの各値は特定の型として認識されデータベースに格納されています。RDBMSと同様に、MongoDB/BSONにはいくつかのデータ型が用意されています。先程みたように、ドキュメント挿入時に明示的に型を定義する必要はありませんが、シェル/ドライバが自動的にデータを適当な型に変換してデータベースに格納します。当然ですが、もしシェル/ドライバが解釈できないデータを挿入しようとしたら何らかのエラーが発生します。

ここで、最も使われるデータ型<sup id="a2">[2](#f2)</sup>を紹介します

<p id="table1" style="text-align:center">**表1**</p>

| 型の種類 | 具体例 |
| :-----: |:-----------|
| 数字     | `{"x": 2.71}` | 
| 文字列   | `{"x": 'foo'}` | 
| 日時     |`{"x": new Date()}` | 
| ブーリアン | `{"x" : true}`|
| 配列      |`{"x": ["a", 2, "c"]}` |
| 埋め込みドキュメント | `{"x" : {"foo" : "bar"}}` | 
| Object ID | `{"x": ObjectId("589810d9c75c4997748e77ef")}`



<sup id="f2">**[2]** 上記の表のもの以外に、正規表現型やバイナリ型など、MongoDBには様々な型があります。詳しくは[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/bson-types/)を見てくだい。[↩](#a2)</sup>

数字用の型はいくつかあるのですが、シェル上では数字は浮動小数点(`Double` - Pythonでいう`float`)として扱われます。しかし、整数を扱うことも可能で、Pythonドライバを使う場合はPythonの`int`は正しく整数(`Int64`）として扱われます。

文字列は任意のUTF-8で符号化されたものを扱うことができます。なお、MongoDB/BSONではUTF-8以外の文字符号化方式を使うことが不可能です。

日時はUNIXエポックからのミリ秒の数を表す整数として保存されます。また、時間帯サポートがないので、すべての日付はUTCとして保存されます。マイクロ秒・ナノ秒単位の時間を保存する必要がある場合は違う`Int64`等、違う型を使うことが必要です。

ブーリアン型は真実(`true`)か偽(`false`)を保存します。

配列では任意の型の複数の値のものを保存するのに使います。MongoDBの大きな特徴の一つはやはり配列を[第一級オブジェクト](https://ja.wikipedia.org/wiki/第一級オブジェクト)(*first-class object*)として扱っていることです。これは非常に便利でRDBMSにはない特徴です。また、MongoDBの配列はPythonでいう`list`と同様に異なる型の値を同じ配列に納めることが可能です（上の表の例を参照）。

埋め込みドキュメント(*embedded document*)とは他のドキュメントのキーの値として使われるドキュメントのことです。埋め込みドキュメントを使うことでRDBMSのようにデータ構造をフラットなものに限定する必要がありません。先程データベースに挿入した2つ目のツイートの`user_mentions`フィールドは実は一つの要素の配列の中に一つの埋め込みドキュメントが入っていました。このように、配列や埋込ドキュメントを使うことで、複雑な階層構造のデータをそのまま保存することができます。

#### `ObjectID`/`_id`フィールドについて

前節で示した[`find`結果](#7.2.4-find)には`_id`フィールドが追加されているのを不思議に思った読者も多いでしょう。実は、MongoDBでは全てのドキュメントには必ずユニークな`_id`フィールドが与えられます。一般的に、MongoDBが自動的にその`ObjectId`を生成しますが、ユーザが明示的に指定することもできます。

`ObjectId`の生成方法は複数のマシン・プロセスがが同時に大量のドキュメントを挿入しても問題が起こらないように設計されています。`ObjectId`の最初の4バイトはUNIX時間、次の3バイトはマシンID、次の2バイトはプロセスID、そして最後の3バイトはランダムに初期化されるカウンタです（下記図を参照）。もし、より単純なカウンタのようなものを使っていれば、多くのクライアントから同時にアクセスがあったときに、データベースがカウンタを管理しきれず、重複`ObjectId`が発生しエラーが発生しかねないため、このような設計になっています。また、詳しくは[7.4.1節](#7.4.1-インデックス)で述べますが、`ObjectId`はすべてのコレクションで自動的に作成されるデフォルトインデックスにも使われます。

![img](https://docs.google.com/drawings/d/1HBZMNvd8WsnoUKzImPRkpIoKwx-rHOp5AuyomIjA2PQ/pub?w=1200)

***

## 7.3 MongoDBの基本操作

本節ではTwitterデータを使って、MongoDBのいわゆるCRUD操作(create, read, update, delete)および集計操作について学びます。それらの操作はすべて**コレクションに対して**行うものです、各操作に対する代表的なメッソドを以下の表示しています。

<p id="table2" style="text-align:center">**表2**</p>

| 操作 | 代表的なメソッド |
| :-----: |:-----------|
| Create  | `db.collection.insertOne(<ドキュメント>, <オプション>)` <br> `db.collection.insertMany(<ドキュメントの配列>, <オプション>)`| 
| Read    | `db.collection.find(<クエリ>, <プロジェクション>)` | 
| Update  | `db.collection.updateOne(<フィルタ>, <アップデート>, <オプション>)` <br> `db.collection.updateMany(<フィルタ>, <アップデート>, <オプション>)` <br> `db.collection.replaceOne(<フィルタ>, <新しいドキュメント>, <オプション>)`| 
| Delete  | `db.collection.deleteOne(<フィルタ>, <オプション>)` <br> `db.collection.deleteMany(<フィルタ>, <オプション>)` <br> `db.collection.drop()`| 

本節ではそれらを順番に詳しく見ていきます。この表2に何回か言及するので、しっかり覚えておくようにしましょう。

その前に、[7.2.4節](#7.2.4-Mongoシェルの基本操作)と同様に、まずは`twitter`データベースを選択し、リセットしましょう。

In [8]:
use twitter

switched to db twitter

In [7]:
db.dropDatabase()

{ "dropped" : "twitter", "ok" : 1 }

### 7.3.1 データの挿入 (Create)



[7.2.4節](#7.2.4-Mongoシェルの基本操作)では下記のように`insert`を使って**1つ**のドキュメントを挿入しました。

In [9]:
db.tweets.insertOne({'created_at': 'Sun Feb 05 06:37:03 +0000 2017',
  'hashtags': [{'text': 'AmericaFirst'},
   {'text': 'Boeing'},
   {'text': 'aircraft'},
   {'text': 'アメリカ第一主義'}],
  'id': 828130298042974208,
  'lang': 'en',
  'source': '<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>',
  'text': 'The tragic reality of #AmericaFirst. #Boeing #aircraft components come from all over the world. #アメリカ第一主義... https://t.co/FDk5lvtkLb',
  'urls': [{'expanded_url': 'http://fb.me/8yFcJ2k7H',
    'url': 'https://t.co/FDk5lvtkLb'}],
  'user_mentions': []})

{
	"acknowledged" : true,
	"insertedId" : ObjectId("592a0ccf48ffa7f5b51f66f2")
}

それ以外に、複数のドキュメントを入れるときはドキュメントを配列にまとめて、`insertMany`を使います。

In [10]:
db.tweets.insertMany([
{'created_at': 'Sun Feb 05 06:35:25 +0000 2017',
  'hashtags': [],
  'id': 828129887214919680,
  'lang': 'en',
  'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
  'text': '@mgsiegler its def more like the San Diego Chargers logo https://t.co/iW9qgwKN47',
  'urls': [{'expanded_url': 'https://munchies.vice.com/en/articles/kurdistans-hottest-new-restaurant-is-trump-fish',
    'url': 'https://t.co/iW9qgwKN47'}],
  'user_mentions': [{'id': 652193,
    'name': 'M.G. Siegler',
    'screen_name': 'mgsiegler'}]},
 {'created_at': 'Sun Feb 05 06:52:06 +0000 2017',
  'hashtags': [],
  'id': 828134084291461120,
  'lang': 'en',
  'text': '.@NYDNOpinions: A college education for Trump on free speech and public safety\xa0” https://t.co/3XCqnBue2D https://t.co/icNHpcScjL',
  'urls': [{'expanded_url': 'http://nydn.us/2kAgod7',
    'url': 'https://t.co/3XCqnBue2D'}],
  'user_mentions': [{'id': 26496560,
    'name': 'NYDN Opinion Team',
    'screen_name': 'NYDNOpinions'}]}
])

{
	"acknowledged" : true,
	"insertedIds" : [
		ObjectId("592a0cd148ffa7f5b51f66f3"),
		ObjectId("592a0cd148ffa7f5b51f66f4")
	]
}

また、データを直接入力しなくても、好きなプログラミング言語から様々な方法でデータを挿入できます。MongoシェルはJavascriptがほぼそのまま使えるため、例えば以下のようにファイルからデータを読み込んで挿入することができます。

In [26]:
var data = JSON.parse(cat("mongo_data/sample_tweets.json"));
db.tweets.insertMany(data)

{
	"acknowledged" : true,
	"insertedIds" : [
		ObjectId("592a108807fa13bd83166e0a"),
		ObjectId("592a108807fa13bd83166e0b"),
		ObjectId("592a108807fa13bd83166e0c"),
		ObjectId("592a108807fa13bd83166e0d"),
		ObjectId("592a108807fa13bd83166e0e"),
		ObjectId("592a108807fa13bd83166e0f"),
		ObjectId("592a108807fa13bd83166e10")
	]
}

また、[7.4.3節](##7.4.3-データのバックアップ・エクスポート)で紹介するCLIツールを使ってデータを挿入することも可能です。

>**注意**: Mongoシェル上では`insertOne`や`insertMany`以外に`insert`というのもありますが、現在のMongo v3.4ではPythonのpymongoをはじめ、多くの言語のドライバでは**非推奨**(deprecated)扱いになっているので、使わない方が無難でしょう。

### 7.3.2 データの検索 (Read)


データを検索するのに`find`メッソドを使いますが、一般的に対象データを指定するために**クエリドキュメント**というパラメータを渡します。**クエリドキュメント**はSQL構文の`where`によく似ており、特定の条件にマッチしたドキュメントのみを取得するのに使います。

[7.2.4節](#7.2.4-Mongoシェルの基本操作)で見たように、最も単純なクエリドキュメントはコレクションの全てのドキュメントにマッチする空ドキュメント{}です（何もを渡さないのと同じです）。また、あるフィールドが特定の値になっているドキュメントのみを検索した場合は`{<field>: <value>}`のようなドキュメントを渡せばいいです。例えば、日本語のツイートだけ検索したい場合は、`{'lang': 'ja'}`を使えばいいです。

In [13]:
db.tweets.find({'lang': 'en'})

{ "_id" : ObjectId("592a0ccf48ffa7f5b51f66f2"), "created_at" : "Sun Feb 05 06:37:03 +0000 2017", "hashtags" : [ { "text" : "AmericaFirst" }, { "text" : "Boeing" }, { "text" : "aircraft" }, { "text" : "アメリカ第一主義" } ], "id" : 828130298042974200, "lang" : "en", "source" : "<a href=\"http://www.facebook.com/twitter\" rel=\"nofollow\">Facebook</a>", "text" : "The tragic reality of #AmericaFirst. #Boeing #aircraft components come from all over the world. #アメリカ第一主義... https://t.co/FDk5lvtkLb", "urls" : [ { "expanded_url" : "http://fb.me/8yFcJ2k7H", "url" : "https://t.co/FDk5lvtkLb" } ], "user_mentions" : [ ] }
{ "_id" : ObjectId("592a0cd148ffa7f5b51f66f3"), "created_at" : "Sun Feb 05 06:35:25 +0000 2017", "hashtags" : [ ], "id" : 828129887214919700, "lang" : "en", "source" : "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text" : "@mgsiegler its def more like the San Diego Chargers logo https://t.co/iW9qgwKN47", "urls" : [ { "expanded_url" : "https://munchies.vice.co

#### プロジェクション

[表2](#table2)にあるように、`find`には**プロジェクション**というもう一つのパラメータを渡せます。プロジェクションはSQLでいう`SELECT`構文のようなもので、検索クエリでマッチしたドキュメントの取得するフィールドを指定するのに使います。
例えば、ツイートの本文**だけ**を取得したい場合は次のようにしてします:

In [25]:
db.tweets.find({}, {'text': 1})

{ "_id" : ObjectId("5928598e003d6923dd6ce7d5"), "text" : "トランプ大統領が招く「憲法の危機」 入国禁止令を停止した裁判官をTwitterで攻撃\n\n困ったちゃん https://t.co/BLWVqXOAAw" }
{ "_id" : ObjectId("592859e7003d6923dd6ce7d6"), "text" : "@GeorgeTakei She's the best! Comedy will help us make it through the Trump shit show" }

デフォルトで`_id`も返されますが、プロジェクションを下記のようにすると`_id`フィールドを省略できます。

In [ ]:
db.tweets.find({}, {'text': 1, '_id': 0})

基本的に一つのプロジェクションではフィールドの明示的な選択(`1`)と明示的な排除(`0`)を混ぜることができません（唯一の例外は先程みた`_id`フィールドです）。少し考えればその理由はわかると思いますが、とにかくどちらかを選択する必要があります。したがって、例えば、次のクエリではエラーが発生します。

In [15]:
db.tweets.find({}, {'hashtags': 0, 'text': 1})

Error: error: {
	"ok" : 0,
	"errmsg" : "Projection cannot have a mix of inclusion and exclusion.",
	"code" : 2,
	"codeName" : "BadValue"
}

しかし、明示的なフィールド選択**だけ**なら可能です:

In [ ]:
db.tweets.find({}, {'hashtags': 1, 'text': 1});

逆に、明示的なフィールドの排除**だけ**なら可能です:

In [ ]:
db.tweets.find({}, {'_id': 0, 'urls': 0, 'hashtags': 0, 'source': 0, 'user_mentions': 0})

#### クエリ演算子

MongoDBの大きな特徴としては「`$`」で始まる**演算子**(operator)です。演算子は様々な用途に使われますが、クエリでよく使う演算子には以下のものがあります。

<p id="table3" style="text-align:center">**表3**</p>

| 演算子 | 説明 |
| :-----: |:-----------|
| `$eq`   | equal, 等しい|
| `$lt`   | less than, 未満|
| `$lte`  | less than or equal, 以下     |
| `$gt`   | greater than, より大きい    |
| `$gte`  | greater than or equal, 以上     |
| `$exists`  | フィールドが存在するかどうか     |
| `$in`  | 与えられた値が配列に含まれているかどうか     |
| `$or`  | 複数のクエリの論理和     |
| `$and`  | 複数のクエリの論理積     |
| `$not`  | 論理積否定     |
| `$regex`  | 正規表現     |


実は先程紹介した`{<field>: <value>}`は`{<field>: {$eq: <value>}}`を意味するただの[シンタックスシュガー](https://ja.wikipedia.org/wiki/糖衣構文)でした。

例えば、言語が英語で、ツイートの`id`が`828130464015003648`より大きいツイートが欲しい場合は上記の`$gt`演算子を使って、下記のように検索できます（プロジェクションも同時に渡しています）。

In [ ]:
db.tweets.find({'lang': 'en', 'id': {$gt: 828130464015003648}},
               {'_id': 0, 'text': 1, 'id': 1})

クエリでもプロジェクションでもそうですが、複数のフィールドを指定することで、それらの論理積（[`AND`](https://ja.wikipedia.org/wiki/論理積)）をとって検索することになります。例えば、先程の検索で使ったクエリドキュメントの場合は`lang`が`en`**かつ**`id`が`828130464015003648`より大きいものみが返されます。

他の演算子を使うと、様々な検索ができます。例えば、`$exists`を使うことでフィールドの有無を検索条件にすることもできます:

In [ ]:
db.tweets.find({'source': {$exists: false}}, {'_id': 0})

`$in`は特定の値が配列の中に含まれているもののみを検索するのに使います。`$in`で使う配列の要素が複数の場合は論理和([`OR`](https://ja.wikipedia.org/wiki/論理和))をとって検索します。

In [34]:
db.tweets.find({'hashtags.text': {$in: ['AmericaFirst', 'tax']}},
                {'_id': 0, 'text': 1})

{ "text" : "The #border adjustment #tax #proposal could rise #gasoline #price by 30 cents. ~via @OilandEnergy https://t.co/3VfUehiR59\n#WTI #Trump #Brent" }

上記の検索では、「`AmericaFirst`」もしくは「`tax`」が含まれているツイートが返されます
。ここで、クエリに`hashtags.text`を使うことで、`hashtags`の値に含まれる**埋め込みドキュメント**のフィールドを直接検索することができてしまいます。この機能は便利ですが、その反面、フィールド名に「`.`」を使うことが禁止されています。


なお、`$in`の対象フィールド（上記の例で`hashtags`)の値は配列である必要がないので、以下のような検索も有効です:

In [ ]:
db.tweets.find({'lang': {$in: ['es', 'ja']}},  {'_id': 0, 'text': 1})

また、文字列の検索<sup id="a2">[2](#f2)</sup>をしたい場合は、正規表現演算子の`$regex`を使うと便利です:

<sup>**[2]** テキストインデクスを利用することで`$text`演算子でより高度な文字列検索が可能です。詳しくは[7.4.1節](#7.4.1-インデックス)および[公式ドキュメンテーション](https://docs.mongodb.com/manual/text-search/)を見てください[↩](#a2)</sup>

In [ ]:
db.tweets.find({'text': {$regex: /america/i}},  {'_id': 0, 'text': 1})

>**注意**: Mongoシェルで使う正規表現はいわゆるPerl用のもの(PCRE)でPythonで使うものと少しだけ異なります。なお、Pythonドライバを使う場合はPythonの`re.complie`等を使うことでPythonの正規表現も問題なく使います。

ここで紹介したのはよく使う基礎的な演算子ですが、MongoDBには他にたくさんの演算子があります。クエリ用演算子の詳細については[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors)を見てみてください。

**<練習問題1>**

`hashtags`フィールドが存在し、かつそれがから配列でないツイートを検索せよ。

In [37]:
db.tweets.find({'hashtags':{$ne:[]}})

{ "_id" : ObjectId("5928598e003d6923dd6ce7d5"), "created_at" : "Sun Feb 05 06:28:27 +0000 2017", "id" : 828128132402073600, "lang" : "ja", "text" : "トランプ大統領が招く「憲法の危機」 入国禁止令を停止した裁判官をTwitterで攻撃\n\n困ったちゃん https://t.co/BLWVqXOAAw" }
{ "_id" : ObjectId("592859e7003d6923dd6ce7d6"), "created_at" : "Sun Feb 05 06:48:27 +0000 2017", "id" : 828133165654945800, "lang" : "en", "text" : "@GeorgeTakei She's the best! Comedy will help us make it through the Trump shit show", "user_mentions" : [ { "id" : 237845487, "name" : "George Takei", "screen_name" : "GeorgeTakei" } ] }
{ "_id" : ObjectId("592a108807fa13bd83166e0d"), "created_at" : "Sun Feb 05 06:31:00 +0000 2017", "hashtags" : [ { "text" : "border" }, { "text" : "tax" }, { "text" : "proposal" }, { "text" : "gasoline" }, { "text" : "price" }, { "text" : "WTI" }, { "text" : "Trump" }, { "text" : "Brent" } ], "id" : 828128775548260400, "lang" : "en", "source" : "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>

#### 7.3.2.1 カーソル操作

[7.2.3節](#7.2.3-MongoDBの仕組み)で述べたように、MongoDBに対して検索コマンドを送るときに返されるのは**カーソル**であると述べました。`find`コマンドはカーソルを返しますが、そのカーソルの実際の実行・データ取得・表示は必要になるまで行われません<sup id="a3">[3](#f3)</sup>。Mongoシェルをではカーソルは即時に実行され、データの一部が表示してされますが、Pythonドライバではカーソルに対して何らかの操作(例えば、`list()`をコールする等）をしないと、実際のデータ取得・表示は行われません。

<sup>**[3]** これを[遅延評価](https://ja.wikipedia.org/wiki/遅延評価)(lazy evaluation)といい、関数型言語でよく使用されるテクニックです。[↩](#a3)</sup>

これまでは**コレクションに対して**行う操作を紹介してきましたが、ここでは**カーソルに対して**行ういくつかの便利な操作を以下に紹介します。

<p id="table4" style="text-align:center">**表4**</p>

| メソッド | 説明 |
|:-----------|:-----------|
| `cursor.sort(<ソート>)` | 検索結果を整列する  |
| `cursor.count()` | 検索結果のドキュメントの数を返す　|
| `cursor.limit(<数>)` | 返されるドキュメント数を制限する |
| `cursor.skip(<数>)` | 返されるを<数>だけスキップする |

例えば、各ツイートの`id`だけに注目すると、

In [38]:
db.tweets.find({}, {'_id': 0, 'id': 1})

{ "id" : 828128132402073600 }
{ "id" : 828133165654945800 }
{ "id" : 828128132402073600 }
{ "id" : 828133165654945800 }
{ "id" : 828132464015003600 }
{ "id" : 828128775548260400 }
{ "id" : 828128399092699100 }
{ "id" : 828126283049664500 }
{ "id" : 828121664869134300 }

整列されていないことがわかります。ここで、ソートドキュメントを使って以下のように整列できます:

In [39]:
db.tweets.find({}, {'_id': 0, 'id': 1}).sort({'id': 1})

{ "id" : 828121664869134300 }
{ "id" : 828126283049664500 }
{ "id" : 828128132402073600 }
{ "id" : 828128132402073600 }
{ "id" : 828128399092699100 }
{ "id" : 828128775548260400 }
{ "id" : 828132464015003600 }
{ "id" : 828133165654945800 }
{ "id" : 828133165654945800 }

ソートドキュメントにおいて、`1`は昇順、`-1`は降順を意味します。複数のフィールドを組み合わせることも可能です:

In [40]:
db.tweets.find({}, {'_id': 0, 'id': 1, 'hashtags': 1}).sort({'hashtags': -1, 'id': 1})

{ "id" : 828121664869134300, "hashtags" : [ { "text" : "トランプ" }, { "text" : "イラン" } ] }
{ "hashtags" : [ { "text" : "border" }, { "text" : "tax" }, { "text" : "proposal" }, { "text" : "gasoline" }, { "text" : "price" }, { "text" : "WTI" }, { "text" : "Trump" }, { "text" : "Brent" } ], "id" : 828128775548260400 }
{ "id" : 828128132402073600 }
{ "id" : 828133165654945800 }
{ "hashtags" : [ ], "id" : 828126283049664500 }
{ "hashtags" : [ ], "id" : 828128132402073600 }
{ "hashtags" : [ ], "id" : 828128399092699100 }
{ "hashtags" : [ ], "id" : 828132464015003600 }
{ "hashtags" : [ ], "id" : 828133165654945800 }

上記の検索では`hashtags`に対して降順で整列したのち、さらに`id`で昇順で整列しています。

また、上記の検索で、取得されるドキュメント数を制限したい場合は`limit`を使います:

In [41]:
db.tweets.find({}, {'_id': 0, 'id': 1, 'hashtags': 1})
         .sort({'hashtags': -1, 'id': 1})
         .limit(5)

{ "id" : 828121664869134300, "hashtags" : [ { "text" : "トランプ" }, { "text" : "イラン" } ] }
{ "hashtags" : [ { "text" : "border" }, { "text" : "tax" }, { "text" : "proposal" }, { "text" : "gasoline" }, { "text" : "price" }, { "text" : "WTI" }, { "text" : "Trump" }, { "text" : "Brent" } ], "id" : 828128775548260400 }
{ "id" : 828128132402073600 }
{ "id" : 828133165654945800 }
{ "hashtags" : [ ], "id" : 828126283049664500 }

また、最初のいくつかのドキュメントをスキップしたい場合は`skip`を使えばいいです:

In [ ]:
db.tweets.find({}, {'_id': 0, 'id': 1, 'hashtags': 1})
         .sort({'hashtags': -1, 'id': 1})
         .skip(5)

検索結果のドキュメントの個数だけを知りたい場合は`count`を使います:

In [ ]:
db.tweets.find({'lang': {$in: ['es', 'ja']}}).count()

また、コレクション全体のドキュメント数を知りたい場合は単純に:

In [ ]:
db.tweets.find().count()

なお、`db.tweets.count()`でも同じ結果が得られます。これもまたただののシンタックスシュガーです。

`sort`のようにカーソルに対して行う操作がまたカーソルを返す場合、上記の例のように複数の操作を繋ぎ合わせることができます。しかし、当然ながら`count`のようにカーソルではなく、数値等を返す操作の場合はできません。

今回の小さなデータセットの場合はパフォーマンスのことを意識せずにいろんなフィールドに対してソート操作を行いましたが、通常、ソートをするときには**インデックス**が使われます。大きなデータセットをインデックスなしでソートしようとしたら、非常に時間がかかるかエラーが発生します。[7.4.1節](#7.4.1-インデックス)で詳しく述べますが、検索やソートでよく使うフィールドについてはインデックスを作る必要があることを覚えておいてください。

### 7.3.3 データの更新 (Update)

データの更新を行うとき、1)あるドキュメントを丸ごと他のドキュメントと入れ替える、2)一つもしくは複数のドキュメントの特定のフィールドをに何らかの操作をする、の2つのパターンがあります。1)では`replaceOne`を使い、2)では`updateOne`もしくは`updateMany`を使います。各メソッドのシグネチャは[表2](#table2)を参照してください。

#### ドキュメントの入れ替え

今回のTwitterデータセットにはスペイン語のツイートが一つだけあります:

In [42]:
db.tweets.find({'lang': 'es'}, {'text': 1, '_id': 0})

{ "text" : "Me ha gustado un vídeo de @YouTube de @thedracergx (https://t.co/L1UwJwXxyu - LORD PEÑA NIETO (S2): Attack on Trump - Anime" }

あまり現実的な例ではないですが、仮にそのスペイン語ツイートを`{'foo': 'bar'}`というドキュメントと入れ替えたい場合、下記のコマンドを実行すればいいです。

In [43]:
db.tweets.replaceOne({'lang': 'es'}, {'foo': 'bar'})

{ "acknowledged" : true, "matchedCount" : 1, "modifiedCount" : 1 }

先程の`find`コマンドをもう一回実行すると、スペイン語のツイートがなくなっていることが確認できます。一方で、次のコマンドで、新しく先程の`{'foo': 'bar'}`ドキュメントが挿入されたことがわかります。

In [44]:
db.tweets.find({'foo': {$exists: true}})

{ "_id" : ObjectId("592a108807fa13bd83166e0f"), "foo" : "bar" }

このような操作はそもそもなぜ必要なのかと思われるかもしれませんが、実は各ドキュメントに対してMongoDBだけでは対処しきない操作をしたい場面では便利です。例えば、ツイートの本文をGoogle翻訳APIを使って翻訳したい場合はPython等からMongoDBからデータを取得し、翻訳やその他の操作をしてから元のドキュメントと入れ替えたいときには、`replaceOne`を使うと削除操作と挿入操作を同時にできてしまうので一石二鳥です。

#### フィールド操作と更新演算子

一般的に、ドキュメントを丸ごと入れ替えるより、ドキュメントの特定のフィールドだけを変更することが多いでしょう。Twitterの場合はツイートがリツイートされたら、そのリツイートカウントに関するフィールドだけを変更したい例がわかりやすいでしょう。

MongoDBでは、こういった更新操作のための演算子をいくつか用意しています。本節では下記の**表5**にある3つだけを紹介しますが、他にたくさんあるのでぜひ[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/update/)を見てみてください。

<p id="table5" style="text-align:center">**表5**</p>

| 演算子 | 説明 |
| :-----: |:-----------|
| `$set`   | フィールドの値を設定 |
| `$inc`   | 数値を増減 |
| `$push`  | 配列に値を挿入 |


- **`$set`**

下記のツイートでは日本語のハッシュタグは入っているにもかかわらず、言語が英語(`en`)になっています。

In [45]:
db.tweets.find({'hashtags.text': {$in : ["アメリカ第一主義"]}})

もし、このツイートの言語を日本語に変更したい場合は`$set`を下記のように使えばいいです:

In [46]:
db.tweets.updateOne({'hashtags.text': {$in : ["アメリカ第一主義"]}},
                    {$set: {'lang': 'ja'}})

{ "acknowledged" : true, "matchedCount" : 0, "modifiedCount" : 0 }

もう一回`find`を使うと、確かに言語の値が変更されていることがわかります。また、`_id`が変わっていないので、前と同じドキュメントであることも確認できます。

In [ ]:
db.tweets.find({'hashtags.text': {$in : ["アメリカ第一主義"]}},
                {'text': 1, 'lang': 1})

今回は一つだけのフィールドを変更しましたが、複数のフィールドを同時に変更することも可能です。

- **`$inc`**

先程のリツイートのカウントの変更について述べましたが、具体的には下記のツイートの例を考えましょう:

In [53]:
db.tweets.find({'text': {$regex : 'RT'}},
               {'_id': 0, 'text': 1, 'retweet_count': 1, 'hashtags': 1})

{ "text" : "RT @HuffPostJapan: 【 #トランプ 大統領】ミサイル実験を実行した #イラン に新たな経済制裁 でも1年前の制裁と内容ほぼ同じ\nhttps://t.co/VpvVlSzB80", "hashtags" : [ { "text" : "トランプ" }, { "text" : "イラン" }, { "text" : "ミサイル" } ], "retweet_count" : 8 }

仮にこのツイートがリツイートされて、`retweet_count`に1を足したい場合は以下のコマンドを実行すればいいです:

In [50]:
db.tweets.updateOne({'text': {$regex : 'RT'}},
                    {$inc : {'retweet_count': 1}})

{ "acknowledged" : true, "matchedCount" : 1, "modifiedCount" : 1 }

先程の`find`コマンドをもう一回実行すると`retweet_count`が増えていることがわかります。

このよう場合は`find`で現在の`retweet_count`を取得して、`$set`でそれを変更することも可能ですが、`$inc`の方が一回の操作で済みます。また、`updateMany`で複数のドキュメントを同時に更新するときは`find`+`$set`より遥かに便利であることが想像できるかと思います。

- **`$push`**

Twitter上でツイート本文を変更することはできないですが、仮にできたとして、先程のツイートの本文をの「ミサイル」という単語が「#ミサイル」というハッシュタグに変更されたとしましょう。その場合は`hashtags`フィールドの配列に`{'text': 'ミサイル'}`を追加することになるのですが、そのような操作は下記のように`$push`コマンドを使います: 


In [52]:
db.tweets.updateOne({'text': {$regex : 'RT'}},
                    {$push: {'hashtags': {'text': 'ミサイル'}}})

{ "acknowledged" : true, "matchedCount" : 1, "modifiedCount" : 1 }

先程の`find`コマンドをもう一回実行すると`hashtags`の配列に新しい要素が追加されたことがわかります。

#### Upsert

`updateOne`と`updateMany`の3つ目のパラメータはオプション用([表2](#table2)を参照）ですが、最も使うオプションの一つは`upsert`というものです。Upsertとは「update」と「insert」の造語でそれをオンにすると、検索クエリにマッチしたドキュメントがあれば場合はそのドキュメントを更新し、無ければドキュメントを挿入します。

これまでツイッターデータを使っていましたが、今回はある会社の社員名簿を扱っているとしましょう。そして、「Taro Mongo」という社員のメールアドレスを設定するため、以下のコマンドを考えます。

In [54]:
db.contacts.updateOne({'name': 'Taro Mongo'},
                      {'$set': {'email': 'taro.mongo@mongodb.com'}})

{ "acknowledged" : true, "matchedCount" : 0, "modifiedCount" : 0 }

ここで、シェルの出力を見ると、フィルタ`{'name': 'Taro Mongo'}`にマッチしたドキュメントがなかったため、更新が行われなかったことがわかります。とろこが、`{'upsert': true}`を使うと:

In [55]:
db.contacts.updateOne({'name': 'Taro Mongo'},
                      {'$set': {'email': 'taro.mongo@mongodb.com'}},
                      {'upsert': true})

{
	"acknowledged" : true,
	"matchedCount" : 0,
	"modifiedCount" : 0,
	"upsertedId" : ObjectId("592a14b1ba2311f8d6976561")
}

ドキュメントが挿入されます。それを確認すると、フィルタとアップデートドキュメントを合成したようなドキュメントになっていることが分かります。

In [56]:
db.contacts.find()

{ "_id" : ObjectId("592a14b1ba2311f8d6976561"), "name" : "Taro Mongo", "email" : "taro.mongo@mongodb.com" }

#### 複数のドキュメントを扱う場合

これまでは`updateOne`だけを使って、常に一つだけのドキュメントを更新する例を示してきましたが、`updateMany`は`updateOne`とほぼ全く同じように使います。例えば、Twitterデータセットにすべてのツイートに既読(`read`)フィールドを追加したい場合は以下のようにすればいいです:

In [58]:
db.tweets.updateOne({}, {$set: {'test': true}})

{ "acknowledged" : true, "matchedCount" : 1, "modifiedCount" : 1 }

>**注意**: Mongoシェル上では`updateOne`や`updateMany`以外に単に`update`というのもありますが、現在のMongo v3.4ではPythonのpymongoをはじめ、多くの言語のドライバでは**非推奨**(deprecated)扱いになっています。また、使い方によっては`update`が`replaceOne`のように振る舞ってしまう場合があり、紛らわしいので、`insert`と同様に使わない方が無難でしょう。

### 7.3.4 データの削除 (Delete)

CRUD操作の最後の「delete」ですが、CRUDの4操作の中最も単純なものです。`deleteOne`と`deleteMany`がとるパラメータはフィルタのみです。例えば、全ての日本語のツイートを削除コマンドを実行すると:

In [60]:
db.tweets.deleteMany({'lang': 'ja'})

{ "acknowledged" : true, "deletedCount" : 3 }

念のため、日本語ツイートの数を確認すると:

In [61]:
db.tweets.find({'lang': 'ja'}).count()

0

空フィルタで`deleteMany`を実行すれば、そのコレクションのすべてのドキュメントが削除されますが、単にコレクションを丸ごと削除したい場合は実は`drop`というメッソドの方が早いです。[表2](#table2)にあるように、`drop`はパラメータを取らないので、今までのツイートをすべて消すには以下のコマンドを実行します:

In [62]:
db.tweets.drop()

true

また、`drop`を使う場合、ドキュメントだけではなく、コレクションに付随されるインデックス等のメタデータもすべて削除されます。

>**注意**: Mongoシェル上では`deleteOne`や`deleteMany`以外に単に`remove`というのもありますが、現在のMongo v3.4ではPythonのpymongoをはじめ、多くの言語のドライバでは**非推奨**(deprecated)扱いになっていますので、`insert`と同様に使わない方が無難でしょう。

### 7.3.5 データの集計

MongoDBで集計処理を行う方法は3つあります。

1. **単一目的操作**:
これまでみてきたような`db.collection.count()`等の一つのコレクションのみを操作対象とする簡易集計です。便利な方法ですが、その反面、高度な集計操作には向いていません。
1. **Aggregationフレームワーク**:
UNIXと似たようなパイプラインの概念を用いて、ドキュメントに対して順番にいくつかの操作を行い、最終的に集計ドキュメントを出力します。SQLでいうGROUP BYやWHERE構文に相当します。検索と更新と同じように、特殊な演算子を使って操作します。
1. **MapReduce機能**:
Map関数およびReduce関数を独自に定義し，集計処理を行う方法です。集計パイプラインではできないような複雑な集計処理を行うために使用されます。直接Javascriptの関数を書く必要があるのと、上級者向けであることから、本節では省略します。

本節では集計パイプラインを中心に紹介します。

#### データベース再構築

[7.3.4節](#7.3.4-データの削除-&#40;Delete&#41;)でTwitterデータセットのデータをすべて削除してしまいましたが、本節ではツイートデータをファイルから挿入します。ターミナルを開いて以下のコマンドを実行してください
```
$ cd /root/userspace/chapters
$ mongoimport --db twitter --collection tweets --file mongo_data/tweets.json --host mongo1
```

以下のコマンドを実行して、データが正しく挿入されたことを確認してください。結果が**576**であればデータ次に進んでください。

In [5]:
use twitter

switched to db twitter

In [6]:
db.tweets.count()

586

なお、MongoDBのバックアップツール等に関しては[7.4.3節](#7.4.3-データのバックアップ・エクスポート)で説明します。


#### 集計パイプラインの概要

Aggregationフレームワークでは、コレクション内のドキュメントを変換および結合を繰り返すことで、欲しい集計データを取得します。 フィルタリング、プロジェクション、グループ化、並べ替え、リミット、スキップ等の変換・結合操作をパイプラインのようにつないでいくことで処理を行います。後ほど紹介しますが、これらの変換・結合操作を行うには**Aggregationフレームワーク演算子**を使います。

たとえば、Twitterデータセットで最も多く使われた言語を知りたい場合、以下のパイプラインを考えます:
1. 各ツイートの言語を取得
1. ツイートを言語毎にグループ化して各言語のツイートの数を数える
1. ツイートが多い言語の順に並び替える
1. 上位5言語のみを表示

上記の各操作は集計パイプラインにおける一つの**ステージ**に相当します。各ステージでデータに何らかの変換・結合操作を行い、その結果を次のステージの入力として渡します。各ステージの入力ドキュメント数と出力ドキュメント数は必ずしも一致する必要がないです。一般的にドキュメント数を減らしていきますが、ドキュメント数が増えるステージもありえます。

各ステージは一つだけの**ステージ演算子**(stage operator)をフィールドに持ち、そのフィールドの値に**パイプライン表現**と呼ばれる特殊なドキュメントを持ちます。パイプライン表現ではこれまでみてきたようなクエリドキュメントやプロジェクションドキュメントに似たようなドキュメントが使われます。

それでは、上記の4ステップの操作をAggregationフレームワークで書くと以下のようになります:

In [7]:
db.tweets.aggregate([{$project: {"lang": 1}},
                     {$group: {"_id": "$lang", "count" : {"$sum" : 1}}},
                     {$sort: {"count": -1}},
                     {$limit: 5}
])

{ "_id" : "en", "count" : 428 }
{ "_id" : "fr", "count" : 87 }
{ "_id" : "ja", "count" : 57 }
{ "_id" : "es", "count" : 4 }
{ "_id" : "und", "count" : 3 }

これで最も多い言語は英語で421ものツイートがあることがあります。

また、例えば、英語のツイートの中でハッシュタグの数毎にツイートをグループ化して、グループ毎の合計ツイート数を求めたい場合は以下のようになります:

In [10]:
db.tweets.aggregate([{$match: {"lang": "en"}},
                     {$project: {"num_hashtags": {$size: "$hashtags"}}},
                     {$group: {"_id": "$num_hashtags", "count" : {$sum : 1}}},
                     {$sort: {"_id": 1}}
                    ])

{ "_id" : 0, "count" : 362 }
{ "_id" : 1, "count" : 33 }
{ "_id" : 2, "count" : 9 }
{ "_id" : 3, "count" : 12 }
{ "_id" : 4, "count" : 6 }
{ "_id" : 5, "count" : 4 }
{ "_id" : 8, "count" : 2 }

英語のツイートの中でその大半にあたる357ツイートにはハッシュタグが付いていないことがわかります。

上記の2つの例でいくつかの新しい演算子が登場したので、順番に説明していきます:

- **`$project`**<br>
データ検索におけるプロジェクションドキュメントに似ていますが、検索のように単純にフィールドを選ぶだけではなく、フィールドを改名したり、**表現演算子**を使っていくつかの操作を加えたりできます。`$<フィールド名`>という表記を使うことで、入力ドキュメントにおけるそのフィールドの値に対して表現演算子の操作を行うことができます。<br>
具体的には上記の1つ目の例では、単純なフィールド選択しかしていませんが、2つ目の例では、`{$project: {"num_hashtags": {$size: "$hashtags"}}}`で`num_hashtags`という新しいフィールドを作り、その中身を`$hashtags`の配列の長さにしています。他に様々な数字操作や文字列操作が可能です（[⇗公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/aggregation/#expression-operators))。
<br><br>
- **`$group`**<br>
SQLでいう`GROUP BY`に非常に似ており、`_id`フィールドで指定した（一つもしくは複数の）フィールドに対してグループ化を行った上で、定義する他の新フィールドで**アキュムレータ**と呼ばれる種類の演算子を使って、集計操作を行います。上記の2つの例では、`count`という新フィールドを定義し、（ツイート毎に1を足すことで）ツイートの合計数を求めています。他に下記の例で示す配列に操作を行ったり、最大値・最小値等を求めたりなど、様々な操作が可能です。
<br><br>
- **`$sort`**<br>
[7.3.2.1節](#7.3.2.1-カーソル操作)でみた`sort`カーソルメッソドと同じ操作です。`1`で昇順、`-1`で降順にドキュメントを整列します。
<br><br>
- **`$limit`**<br>
[7.3.2.1節](#7.3.2.1-カーソル操作)でみた`limit`カーソルメッソドと同じ操作です。次のステージの渡すドキュメント数を指定した数だけに制限します。
<br><br>
- **`$match`**<br>
データ検索におけるクエリドキュメントとほぼ同じで、クエリで使える演算子もそのまま使えます。上記の2つめの例で、次のステージに渡すドキュメントを言語が英語であるものだけに制限するのに使っています。集計パイプラインにおいて、次のステージで処理するドキュメント数を減らすために、できるだけ早い段階で`$match`を使うことがよいとされています。
<br><br>
- **`$size`**<br>
配列の要素の数を返します。
<br><br>
- **`$sum`**<br>
数値の和を返します。非数値の値は無視されます。

上記の2つの例では最終的に数値を求めましたが、Aggregationフレームワークは他に様々な用途に使えます。例えば、ハッシュタグを言語毎にグループ化するのは次のように行います:

In [11]:
db.tweets.aggregate([{$project: {"hashtag": "$hashtags.text", "lang": 1}},
                     {$unwind: "$hashtag"},                    
                     {$group: {"_id": "$lang", "hashtags": {$addToSet: '$hashtag'}}},
                    ])

{ "_id" : "de", "hashtags" : [ "Trump", "MarineLyon" ] }
{ "_id" : "fr", "hashtags" : [ "discrimination", "Hollande", "MarineLePen", "LePen", "Macron", "nichols", "Rediff", "larry", "Louvres", "Monde", "fraud", "voter", "Islam", "Trump", "hillary", "AFP", "trump" ] }
{ "_id" : "ja", "hashtags" : [ "TRUMP", "南京大虐殺", "アパホテル", "アメリカ大統領選", "トランプ", "イラン", "LGBT", "中国", "ゴルフ" ] }
{ "_id" : "en", "hashtags" : [ "Zangeneh", "Iran", "RefugeesWelcome", "hungary", "SuperBowl", "MuslimBanprotest", "LGBT", "AmericaLast", "TrumpFirst", "alternativefacts", "KellyAnneConway", "SCOTUS", "tcot", "Islam", "IsabellaLövin", "NeilGorsuch", "BREXIT", "Trumble", "SoCalledPresident", "トランプ", "Australia", "auspol", "AMJoy", "TrumpsAmerica", "Nordstrom", "MuslimBan", "Japan", "Merkel", "climatechange", "proposal", "Empire", "EDM", "WTI", "aircraft", "Trump", "アメリカ第一主義", "soros", "7News", "gasoline", "Boeing", "PJNET", "AmericaFirst", "IvankaTrump", "PeopleOfTheResistance", "border", "Brent", "POTUS", "NGO", "cri

上記の例で新しくが登場した演算子を説明します:

- **`$unwind`**<br>
配列の各ドキュメントを別々のドキュメントに変換します。このステージ操作では一般的に出力ドキュメント数は入力ドキュメント数より多くなる傾向があります。上記の例では、各ツイートの各ハッシュタグを新しいドキュメントに別けるのに使っています。
<br><br>
- **`$addToSet`**<br>
グループ毎に指定したフィールドの値を集合配列にまとめます。数学的な意味での集合なので、重複値は無視され、出力配列における要素の順番をコントロールできます。

Aggregationフレームワークを使うにあたって、パイプラインでエラーが出たら、ステージを一つずつ試していくことでデバッグできますので、下記の練習問題で困ったらそのテクニックを利用してみてください。

集計用演算子は他にたくさんあるので、必要に応じて[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/aggregation/)を参考にしてください。これまで紹介したAggregationフレームワーク演算子を以下の表にまとめました。

<p id="table6" style="text-align:center">**表6: 本節で紹介したAggregationフレームワーク演算子**</p>

| 演算子 | 種類 |
| :-----:|:-----------:|
|`$project`|ステージ演算子|
|`$group`|ステージ演算子|
|`$sort`|ステージ演算子|
|`$limit`|ステージ演算子|
|`$match`|ステージ演算子|
|`$size`|表現演算子|
|`$sum`|アキュムレータ|
|`$unwind`|ステージ演算子|
|`$addToSet`|アキュムレータ|


**<練習問題2>**

データセットに登場するすべてのハッシュタグをunwindし、数の多いものの順位に上位20ハッシュタグを求めよ。

In [34]:
db.tweets.aggregate([{$project:{"hashtag":"$hashtags.text", "lang":"$lang"}}
, {$unwind: "$hashtag"}
, {$group :{"_id":"$hashtag", "count":{$sum:1}}}
, {$sort : {"count":-1}}
, {$limit : 20}
])

{ "_id" : "Trump", "count" : 37 }
{ "_id" : "トランプ", "count" : 19 }
{ "_id" : "AFP", "count" : 12 }
{ "_id" : "MuslimBan", "count" : 12 }
{ "_id" : "SNL", "count" : 10 }
{ "_id" : "Monde", "count" : 8 }
{ "_id" : "イラン", "count" : 5 }
{ "_id" : "trump", "count" : 4 }
{ "_id" : "TrumpBan", "count" : 3 }
{ "_id" : "PresidentBannon", "count" : 3 }
{ "_id" : "auspol", "count" : 3 }
{ "_id" : "TrumpsAmerica", "count" : 3 }
{ "_id" : "POTUS", "count" : 3 }
{ "_id" : "MuslimBanprotest", "count" : 3 }
{ "_id" : "Rediff", "count" : 3 }
{ "_id" : "ゴルフ", "count" : 3 }
{ "_id" : "DNCForum", "count" : 3 }
{ "_id" : "aircraft", "count" : 2 }
{ "_id" : "WTI", "count" : 2 }
{ "_id" : "LGBT", "count" : 2 }

参考程度に、以下の対照表を載せておきます。


<p id="table7" style="text-align:center">**表7: SQL構文/MongoDB演算子の関係**</p>

| SQL構文 | MongoDB演算子 |
| :----- |:-----------|
|`WHERE`	|`$match`|
|`GROUP BY`	|`$group`|
|`HAVING`	|`$match`|
|`SELECT`	|`$project`|
|`ORDER BY`	|`$sort`|
|`LIMIT`	|`$limit`|
|`SUM()`	|`$sum`|
|`COUNT()`	|`$sum`|
|`join`		|`$lookup`|

***

## 7.4 MongoDBのパフォーマンス向上

### 7.4.1 インデックス

インデックスはMongoDBの検索・ソート等の操作が効率的に行われるための目次・索引のようなものです。 インデックスがなければ、MongoDBはどんなに簡単な検索でも、*colection scan*と呼ばれる操作を行わなければなりません。つまり、コレクション内のすべてのドキュメントをスキャンして、検索に一致するドキュメントのみを選択し、検索結果を返すことになります。 検索に適切なインデックスが存在する場合、MongoDBはそのインデックスを使用して、読み込み対象とするドキュメント数を制限することができます。

RDBMSもそうですが、MongoDBではインデックスは内部的に[B-Tree](https://ja.wikipedia.org/wiki/B木)というデータ構造を使っており、探索時間は$O(\log{}n)$です。単純なcollection scanの探索時間は$O(n)$なので、一般的にインデックスを用いた方が検索時間を圧倒的に早くします。ただし、[インデックスの注意点](#インデックスの注意点)で述べる例外もあるので、注意が必要です。

#### インデックスの確認

インデックスはコレクション毎に定義されていますが、これまで使ってきた`tweets`コレクション内のインデックスを確認するには[`getIndexes()`](https://docs.mongodb.com/manual/reference/method/db.collection.getIndexes/)を使います。

In [35]:
db.tweets.getIndexes()

[
	{
		"v" : 2,
		"key" : {
			"_id" : 1
		},
		"name" : "_id_",
		"ns" : "twitter.tweets"
	}
]

まだ明示的にインデックスを作成していないので、コレクションにあるインデックスは自動的に作成される`_id`に対するインデックスのみです。上記の結果では、`name`はインデクスの名前、`v`はバージョン、`ns`はネームスペース（一般的に`<database>.<collection>`）、`key`はインデクスに含まれるフィールド名を表しています。

インデックスはコレクションのデータと一緒に保存されるので、コレクションをdropしたら(`db.collection.drop()`)そのコレクションに付随されるインデックスも一緒に削除されます。また、ドキュメントを挿入・更新・削除したりするたびに、コレクションにあるインデックスも更新されます。コレクションは最大64インデックスまで持つことが可能です。

#### 検索処理の詳細

検索やソートがどのインデックスをどうやって使っているかを確認するにはきはカーソルメソッドの[`explain()`](https://docs.mongodb.com/manual/reference/method/cursor.explain/)を使います。早速使ってみましょう。

In [ ]:
// db.tweets.find({'lang': 'ja'}).explain("executionStats")

環境によって実行結果は少し変わるかもしれないので、事前に準備した以下のものを使って説明します（前セルの実行結果と似ているはずです）。

検索を行うとき、MongoDBはコレクションにあるインデックスを見て、いくつかの検索戦略を立てます（上記ドキュメントの`queryPlanner`）。それらを検証し、最もいい戦略`winningPlan`を選び、実行にします。今回の場合は使えるインデックスがなかったので、最もいい戦略はcollection scan(`COLLSCAN`)を行うことです。

実際実行された検索の詳細は`executionStats`にまとめられています。それを見ると、実行時間`executionTimeMillis`が56ミリ秒、スキャンさらたドキュメント数`totalDocsExamined`は576（すなわち、コレクションの全ドキュメント）であることが分かります。

今回は取り扱っているのは1MB程度のデータセットなので、インデックスの必要性は低いと言えるかもしれませんが、例えばデータが10GB程度になれば、インデックスなしだと上記の検索が**数十秒**かかると考えられます。MongoDBは固定スキーマがないなど、比較的自由な開発が可能ですが、その反面、インデックスを事前にしっかりとした設計を行わないと期待するパフォーマンスを得ることが難しくなってしまいます。

>**注意**: MongoDBは一旦collection scanを行うと、次の操作を高速化するために、スキャンされたドキュメントをメモリ上に残してしまいます。したがって、上記の`find`/`explain`をもう一度実行すれば、実行時間`executionTimeMillis`が0もしくはそれに近い値になると考えられます。パフォーマンスを検証する際は検索対象データが**メモリに載っていない**ことを保障するために、適宜に`mongo`インスタンスを再起動したり、データベースをドロップして再構築したりしてください（[⇗7.3.5節](#データベース再構築)）。

#### インデックスの作成

それでは先程の検索をより効率的にするために、[`createIndex`](https://docs.mongodb.com/manual/reference/method/db.collection.createIndex/#db.collection.createIndex)でインデックスを作成しましょう。

In [ ]:
db.tweets.createIndex({'lang': 1})

ここでは、`lang`フィールドに対するSingleインデックスを作成しています。ここでは[ソート](#7.3.2.1-カーソル操作)と同様に`1`は値の順序を表しており、`1`なら昇順、`-1`なら降順となります。

インデックスの作成はコレクションに入れる前でも、入れた後でも可能です。しかし、インデックスがあるコレクションに新しいドキュメントを挿入する際には、すべてのインデックスも更新しなかればならないため、インデックスがある分だけ挿入が少し遅くなります。また、今回の`tweets`コレクションは小さいので、瞬時にインデックスが作成されましたが、大きなコレクションのインデックスを作成する場合は時間がかかるため、インデックス作成している間にコレクションを他のスレッド・プロセスからもアクセス可能に保つ必要がある場合は`background`オプションを使うといいでしょう（[⇗公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/method/db.collection.createIndex/#db.collection.createIndex)）。

それでは、先程の検索をもう一回実行してみましょう（**注意**: 実行前には先程の注意欄のことを行ってください）。

In [ ]:
// db.tweets.find({'lang': 'ja'}).explain("executionStats")

ここでも、環境によって実行結果は少し変わるかもしれないので、事前に準備した以下のものを使って説明します。

今回は`queryPlanner`を見てみると、`winningPlan`がいきなり`COLLSCAN`ではなく、`IXSCAN`すなわち、インデックスを使ったスキャンを行っていることがわかります。また、`executionStats`をみると、実行時間が前回の56ミリ秒から1ミリ秒に減っており、またスキャンさらたドキュメント数`totalDocsExamined`は前回の576から55に減っていることが分かります。

今回は最も単純なSingleインデックスを使いましたが、MongoDBには以下の6種類のインデックスがあります。

<p id="table8" style="text-align:center">**表8: インデックスの種類**</p>

| インデックス | 説明 |
| :----- |:-----------|
|[Single](https://docs.mongodb.com/manual/core/index-single/)|一つのだけのフィールドに対するインデックス。<br>トップレベルだけでなく、埋め込みドキュメント内のフィールドもインデックス可能。|
|[Compound](https://docs.mongodb.com/manual/core/index-compound/)|複数のフィールドに対するインデックス。<br>順序があるため、`{a: 1, b: 1}`と`{b: 1, a: 1}`ではインデックスの構造が異なる。|
|[Multikey](https://docs.mongodb.com/manual/core/index-multikey/)|配列の要素に対するインデックス。|
|[Geospatial](https://docs.mongodb.com/manual/applications/geospatial-indexes/)	|[GeoJSON](http://geojson.org/)等の地理空間データを含むフィールドに対するインデックス。<br>`$near`等の[地理空間用のクエリ演算子](https://docs.mongodb.com/manual/reference/operator/query-geospatial/)を使った検索のときに使われる。|
|[Text](https://docs.mongodb.com/manual/core/index-text/)|自然言語に対する全文検索用インデックス。<br>2017年2月現在は日本語には[対応していません](https://docs.mongodb.com/manual/reference/text-search-languages/#text-search-languages)。|
|[Hased](https://docs.mongodb.com/manual/core/index-hashed/)|指定されたフィールドの値のハッシュ値を保存するインデックス。<br>主に[シャーディング](https://docs.mongodb.com/manual/sharding/)環境において、各クラスタの負荷を均一化するために使われています。|

ここで、Multikeyインデックスの例として、各ツイート内のハッシュタッグに対するインデックスを作りましょう。

In [ ]:
db.tweets.createIndex({'hashtags.text': 1})

上記のインデックスがSingleインデックスに見えるかもしれませんが、各ツイートの`hashtags`フィールドが配列であるため、Multikeyインデックスと分類されます。Singleインデックスでは、コレクション内の各ドキュメントはインデックスされたフィールドの値と1対1の関係にあるのに対し、Multikeyインデックスでは各ドキュメントはインデックスされたフィールドの値と1対$n$の関係にあります。今回のデータセットで説明すると、一つのツイートが同時に日本語と英語(すなわち、`lang`が**同時に**`ja`かつ`en`）であることは不可能ですが、同じツイートが複数のハッシュタグを含むことは可能です。

配列要素を含むCompoundインデックス（すなわち、Compound兼Multikeyインデックス）を作成する際、配列に対するフィールドは一つまでと制限されています。それはインデックス自体のエントリー数が爆発的に増えないためです。

それでは、`"Trump"`というハッシュタッグを含むドイツ語のツイートを検索しましょう。

In [ ]:
db.tweets.find({'hashtags.text': 'Trump', 'lang': 'de'}, {'user': 0})

ここでは省略しますが、`explain`を使えば、MongoDBが2つのインデックス(`lang`に対するSingleインデックスと`hashtags.text`に対するMultikeyインデックス）を使う`queryPlan`を立てていることがわかります。また実際実行された検索では、`totalDocsExamined`が2つであることも確認できます。

>**注意**: ネット上等でMongoDBは一つの検索・ソートでは基本的に一つのインデックスしか使えないという記述が多く見られますが、v2.6からは複数のインデックスの使用([Index Intersection](https://docs.mongodb.com/manual/core/index-intersection/))が可能になりました。


**<練習問題3>**

上記の検索に対して`explain`を使って、実際どのような検索が行われているかを調べてください。

In [ ]:
//ここでコードを書いたり、セルを追加したりしてください

#### インデックスの属性

インデックス作成時には、インデックスの挙動を変更するためのいくつかのオプションがあります。そららを以下の表にまとめます:

<p id="table9" style="text-align:center">**表9: インデックスの属性**</p>

| 属性 | 説明 |
| :----- |:-----------|
|[Unique](https://docs.mongodb.com/manual/core/index-unique/)|フィールドの値のユニーク性を保障。<br>重複値の挿入を不可にする。|
|[Sparse](https://docs.mongodb.com/manual/core/index-sparse/) |対象フィールドを持っているドキュメントのみをインデックスする。<br>Uniqueと同時に使用可能で、インデックスの大きさを抑えるメリットがある。|
|[Partial](https://docs.mongodb.com/manual/core/index-partial/) |特定のフィールドを持っているドキュメントのみをインデックスする。<br>新しくv3.2に導入され、Sparseインデックスの機能を拡張したもの。<br>現在、SparseよりPartialの使用が推奨される。|
|[TTL](https://docs.mongodb.com/manual/core/index-ttl/)|特定の経過時間もしくは時刻にドキュメントを自動的に削除する。<br>ログデータ等の処理に便利。|

Uniqueインデックスの例の一つは自動的に作成される`_id`に対するインデックスです。なお、重複の値が存在する状態でUniqueインデックスを作ろうとしたら、エラーが出るので、Uniqueインデックスを使用する際は事前に**挿入前**にインデックスを作ることを勧めます。以前は重複値をドロップすることは可能でしたが、v3.0以降それが不可能になっているので、注意が必要です。

**<練習問題4>**

すでに存在するドキュメント（ツイート）の`ObjectId`を使った新しいドキュメントを`tweets`コレクションに挿入してみて、`_id`に対するインデックスが確かにUniqueであることを確認してください。

#### インデックスの注意点

インデックスは、取り扱っているコレクションの小さなサブセットを取得するときに最も効果的です。また、クエリによってはインデックスを使わない方が高速だったりします。一般的に、一つのクエリで取得するコレクションの割合が高ければ高いほど、インデックスの効率が低下します。

インデックスを使うということは、2つの参照を行うことを意味します。一つ目はインデックス上のドキュメントへのポインタの参照で、二つ目はそのポインタからドキュメント自体への参照です。例えば、コレクション内のすべてのドキュメントを取得するようなインデックスを使ったクエリを考えましょう。そのクエリ実行にかかる時間は、collection scanに比べて、先程の一つ目の参照の分だけ増えるので、実はインデックスを使わない方が早いです。

インデックスが性能向上につながるかどうかはドキュメント数、ドキュメントサイズ、検索の種類等、様々な要因に依存します。経験則から、検索クエリがコレクションの30％以上を返している場合は、collection scanの方が早い可能性は高くなるので、しっかり検索のパフォーマンスを調べるのが大事になってきます。

#### インデックスのまとめ

インデックスは奥が深く、データベースのパフォーマンスに直結します。また、インデックスの各種類・属性を正しく使うことで、本来クライアントサイドで実装しなければならない機能をデータベースサイドでできてしまったりします（例えば、TTLインデックスを使ったログ収集、Geospatialインデックスを作ったドキュメント間の距離を測定するなど）。

小さなデートセットならインデックスのことをあまり心配せずに済むかもしれませんが、大量のデータを扱う場合の検索・ソート性能を向上させるには、自分のデータを理解し、適切なインデックス設計を行うことが大変重要です。

### 7.4.2 MongoDBの統計・メットリックス

MongoDBを使って、たくさんのデータを書き込み・読み出したり、インデックスを作ったりしていると、データベースへの負荷や、コレクション・ドキュメント・インデックスがどれぐらいの容量を使っているかが気になります。

そこで、MongoDBにはそれらの情報を調べるためのシェルコマンドやCLIツールがいくつか用意されています。

#### シェルコマンド

- **`db.serverStatus()`**: MongoDBに関する[多く](https://docs.mongodb.com/manual/reference/command/serverStatus/)のメットリックスをまとめたドキュメントを返します。
- **`db.stats()`**: 選択中のデータベースのコレクション数、インデックス数、容量等の基本データを返します。
- **`db.collection.stats()`**: 当該コレクションに関する[多く](https://docs.mongodb.com/manual/reference/command/collStats/#output)のメットリックスをまとめたドキュメントを返します。更に、`{'indexDetails': true}`というオプションを使うと、インデックスに関する詳細なデータも返します。

試しに`db.stats()`を実行してみると、以下のような結果になります。

In [36]:
db.stats()

{
	"db" : "twitter",
	"collections" : 1,
	"views" : 0,
	"objects" : 586,
	"avgObjSize" : 2699.184300341297,
	"dataSize" : 1581722,
	"storageSize" : 753664,
	"numExtents" : 0,
	"indexes" : 1,
	"indexSize" : 40960,
	"ok" : 1
}

上記の`storageSize`を見ると、こちらのデータセットが1MB未満の容量しか使っていないことがわかると思います。

#### **<練習問題5>**

上記に紹介したコマンドを実行し、どのような出力なのかを見てみてください。


#### CLIツール
シェルで実行するコマンド以外に、`mongostat`と`mongotop`というCLIツールがあります

- **`mongostat`**: MongoDBのリアルタイムパフォーマンスデータ（コネクション数、挿入数、検索数等々）を返します。 データベースの負荷が高いと思われるとき等に便利です。 

- **`mongotop`**: MongoDBがコレクション毎に各CRUD操作（挿入、検索、更新、削除）に使っている時間を1秒毎に返します。UNIX系OSの`top`や`htop`に似ています。

MongoDBのメットリクス等に興味がある読者は[公式ドキュメンテーション](https://docs.mongodb.com/manual/administration/monitoring/)を参考にしてください。

### 7.4.3 データのバックアップ・エクスポート

[7.3.5節](#7.3.5-データの集計)でも少し触れましたが、MongoDBのデータをファイルにバックアップ・エクスポートするCLIツールは以下のようなものがあります:

<p id="table9" style="text-align:center">**表9: バックアップ用CLIツール**</p>

| ツール | 説明 |
| :----- |:-----------|
| [`mongoexport`](https://docs.mongodb.com/manual/reference/program/mongoexport/)| テキスト形式(JSON/CSV)のエクスポート用
| [`mongoimport`](https://docs.mongodb.com/manual/reference/program/mongoimport/)| テキスト形式(JSON/CSV)のインポート用
| [`mongodump`](https://docs.mongodb.com/manual/reference/program/mongodump/)| バイナリ形式(BSON)のエクスポート用
| [`mongorestore`](https://docs.mongodb.com/manual/reference/program/mongorestore/)| バイナリ形式(BSON)のインポート用

基本的な使い方は以下の通りです:

- バイナリデータエクスポート
```
mongodump --db <データベース名> --collection <コレクション名> <ディレクトリ>
```

- バイナリデータインポート
```
mongorestore --db <データベース名> --collection <コレクション名> <ディレクトリ>
```

詳しい使い方に関しては表9にリンクを参考にしてください。また、他のバックアップ方法については[公式ドキュメンテーション](https://docs.mongodb.com/manual/core/backups/)を参考にしてください。